In [1]:
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk
from rich import print

In [ ]:
dataset_name = "Hypersniper/riddles_v1"
dataset= load_dataset(dataset_name, split='all')

In [ ]:
dataset

Converting the riddles to multi-step conversation. 

> Work on following models that are already present locally

     - roberta-large
     - llama-2-7b chat
     - codellama-7b-hf
     
> Ask each model to generate 10 more riddles by prompting with 10 riddles

> Answer the synthetic riddles

> Generate response for the follow up

In [3]:
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
import torch

model1_path = '/home/kamal/.cache/huggingface/hub/models--roberta-large/snapshots/716877d372b884cad6d419d828bac6c85b3b18d9/'

"""
roberta = AutoModelForCausalLM.from_pretrained(
    model1_path,
    device_map="auto"
) """ # takes around 2GB of VRAM

'\nroberta = AutoModelForCausalLM.from_pretrained(\n    model1_path,\n    device_map="auto"\n) '

BitsAndBytesConfig(
     load_in_8bit=False,
  
      load_in_4bit=False,
 
       llm_int8_threshold=6.0,

        llm_int8_skip_modules=None,
    
    llm_int8_enable_fp32_cpu_offload=False,
    
    llm_int8_has_fp16_weight=False
    ,
    bnb_4bit_compute_dtype=Non
    e,
    bnb_4bit_quant_type='fp
    4',
    bnb_4bit_use_double_quant=Fa
    lse,
    **kwargs,
)

In [4]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
roberta = AutoModelForCausalLM.from_pretrained(
    model1_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
)  # takes 1.22 GB of VRAM

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [7]:
roberta_tokenizer = AutoTokenizer.from_pretrained(model1_path)

In [9]:
# there is default chat for the model, so it must give output
roberta_tokenizer.default_chat_template

"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"

In [56]:
import random

# get the questions
questions = [q for q in dataset['instruction']]
random.shuffle(questions)  # no need to reassign 
prompt_template = """Below is a riddle. Come up with 10 more.
Output just the riddles. No numbering and don't output anything else"""
simple_request = "Generate 10 riddles that you know"

prompt_riddles = prompt_template + "\n\n".join(questions[0:10])  
# need to use join, list and string dont concatenate

messages_to_model = [{"role":"user", "content":prompt_riddles}]
simple_message = [{"role":"user", "content":simple_request}]

In [ ]:
# roberta_input = roberta_tokenizer.apply_chat_template(messages_to_roberta, return_tensors='pt').to('cuda')
roberta_input = roberta_tokenizer.apply_chat_template(simple_message, 
                                                      return_tensors='pt').to('cuda')
roberta_input

In [28]:
roberta_tokenizer.decode(roberta_input[0])

'<|im_start|>user\nGenerate 10 riddles that you know<|im_end|>\n'

In [16]:
roberta.generation_config

GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 1
}

In [33]:
roberta_output = roberta.generate(
    roberta_input,
    max_new_tokens=50,
    do_sample=True,
    pad_token_id=roberta_tokenizer.eos_token_id
)

In [32]:
output = roberta_output[0][len(roberta_input[0]):]
output

tensor([2], device='cuda:0')

In [21]:
output_text = roberta_tokenizer.decode(output, skip_special_tokens=True)
output_text # There is no output at all. checking why, with a simpler prompt

''

In [35]:
del roberta  # removes the memory handles

In [36]:
torch.cuda.empty_cache()  # garbage collects the memory

#### Working on llama2-7b

In [34]:
llama_path = "/home/kamal/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/c1b0db933684edbfe29a06fa47eb19cc48025e93/"
llama2 = AutoModelForCausalLM.from_pretrained(
    llama_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
) # takes 11GB of VRAM

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
llama_tokenizer = AutoTokenizer.from_pretrained(llama_path)
# print(llama_tokenizer.default_chat_template) # rich's print fails due to tag
llama_tokenizer.default_chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif true == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must 

In [43]:
llama_tokenizer.special_tokens_map

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}

In [42]:
llama_input = llama_tokenizer.apply_chat_template(simple_message,
                                                  return_tensors='pt')
llama_input

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


tensor([[    1,   518, 25580, 29962,  3251,   403, 29871, 29896, 29900,   364,
          2205,   793,   393,   366,  1073,   518, 29914, 25580, 29962]])

In [50]:
llama_output = llama2.generate(
    llama_input,
    max_new_tokens=100,
    do_sample=True,
    pad_token_id = llama_tokenizer.eos_token_id
)

/home/kamal/jupyter_env/lib/python3.11/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [52]:
# llama_output = llama_output[len(llama_input[0]):]
# llama_output
output = llama_tokenizer.decode(llama_output[0],skip_special_tokens=True)
output

'[INST] Generate 10 riddles that you know [/INST]  Sure! Here are 10 riddles that I know:\n\n1. What has keys but can\'t open locks? (Answer: A keyboard)\n2. What starts with an "e" and ends with an "e" but only contains one letter? (Answer: An envelope)\n3. What has a face but no eyes, nose, or mouth? (Answer: A clock)\n4. What can you catch but not throw? ('

In [57]:
# working on getting the riddle based on the 10 riddles input
llama_10_input = llama_tokenizer.apply_chat_template(messages_to_model,
                                                     return_tensors='pt')
llama_10_input

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


tensor([[    1,   518, 25580, 29962, 13866,   338,   263,   364,  2632, 29889,
         16760,   701,   411, 29871, 29896, 29900,   901, 29889,    13,  6466,
           925,   278,   364,  2205,   793, 29889,  1939,  1353,   292,   322,
          1016, 29915, 29873,  1962,  3099,  1683,  2887,  4513,   408,   385,
         26163, 29889,  1094,  6483,   408,   263, 18002, 29889,  2178,   278,
          6989, 29915, 29879, 15100,   508, 29915, 29873,  8206,   372,   701,
         29889,    13,    13,  8439,   338,   385, 12297,   297,  7316, 29889,
         12074,  1304,   297,   777,  5633,   310,   278,  3186,  9826, 29889,
          2193,  6511,  2305,   304,  1074,  1549, 14603, 29889,    13,    13,
          5618, 19514,   541,  2360,  6089, 29973,    13,    13, 29909,  3800,
          1728,   298,   292,   267, 29892,  7714,   470,  1820, 29892,  3447,
         22843,  2578,  3745, 12185,  2629, 29889, 29871,    13,    13,  7789,
          4298, 29891,   408,   263, 15504,   297,  

In [61]:
llama_tokenizer.decode(llama_10_input[0])

"<s> [INST] Below is a riddle. Come up with 10 more.\nOutput just the riddles. No numbering and don't output anything elseAs round as an apple. As deep as a cup. All the king's horses can't pull it up.\n\nThere is an ancient invention. Still used in some parts of the world today. That allows people to see through walls.\n\nWhat asks but never answers?\n\nA box without hinges, lock or key, yet golden treasure lies within. \n\nStealthy as a shadow in the dead of night, cunning but affectionate if given a bite. Never owned but often loved. At my sport considered cruel, but that's because you never know me at all.\n\nI never was, am always to be. No one ever saw me, nor ever will. And yet I am the confidence of all, To live and breath on this terrestrial ball. \n\nI have a hundred legs, but cannot stand. I have a long neck, but no head. I cannot see. I'm neat and tidy as can be.\n\nA man is found hanging dead from the ceiling of a room. The room’s dimensions are 15 x 15 x 15. The man is on

In [58]:
llama_10_output = llama2.generate(
    llama_10_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=llama_tokenizer.eos_token_id
)

/home/kamal/jupyter_env/lib/python3.11/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [62]:
output_10 = llama_tokenizer.decode(llama_10_output[0], skip_special_tokens=True)
output_10

"[INST] Below is a riddle. Come up with 10 more.\nOutput just the riddles. No numbering and don't output anything elseAs round as an apple. As deep as a cup. All the king's horses can't pull it up.\n\nThere is an ancient invention. Still used in some parts of the world today. That allows people to see through walls.\n\nWhat asks but never answers?\n\nA box without hinges, lock or key, yet golden treasure lies within. \n\nStealthy as a shadow in the dead of night, cunning but affectionate if given a bite. Never owned but often loved. At my sport considered cruel, but that's because you never know me at all.\n\nI never was, am always to be. No one ever saw me, nor ever will. And yet I am the confidence of all, To live and breath on this terrestrial ball. \n\nI have a hundred legs, but cannot stand. I have a long neck, but no head. I cannot see. I'm neat and tidy as can be.\n\nA man is found hanging dead from the ceiling of a room. The room’s dimensions are 15 x 15 x 15. The man is only 6

In [65]:
del llama2
torch.cuda.empty_cache()  # this time the model is not offloadin
# restarting the server to release the memory

### Llama observation

- Llama provides the output of 10 riddles

- Llama encoding and decoding is working same as roberta models

In [6]:
import torch

code_llama_path = "/home/kamal/.cache/huggingface/hub/models--codellama--CodeLlama-7b-hf/snapshots/bc5283229e2fe411552f55c71657e97edf79066c/"
code_tokenizer = AutoTokenizer.from_pretrained(code_llama_path)
codellama = AutoModelForCausalLM.from_pretrained(
    code_llama_path,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16
)  # takes around 11.5GB of VRAM
# with 4-bit quantization 5GB of VRAM is consumed

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
code_tokenizer.default_chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must

In [8]:
simple_request = "Generate 10 riddles that you know"

simple_message = [{"role":"user", "content":simple_request}]

In [12]:
code_request = "Generate 10 dictionary related problems"

code_message = [{"role":"user", "content":code_request}]

In [9]:
code_input = code_tokenizer.apply_chat_template(simple_message,return_tensors='pt').to("cuda")

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


In [13]:
code_input = code_tokenizer.apply_chat_template(code_message,return_tensors='pt').to("cuda")

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


In [14]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id
) 

In [17]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    temperature=0.2
) 

In [21]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    # do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    # temperature=0.2
)

In [23]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    # do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    # temperature=0.2
    repetition_penalty=0.5
)

In [28]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    # temperature=0.2
    repetition_penalty=0.5,
    top_p=10,
    top_k=20
)

In [29]:
output = code_output[0][len(code_input[0]):]
code_tokenizer.decode(output, skip_special_tokens=True)

'ayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaSSLayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaSSLayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaSSLayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaSSLayaayaSSLayaSSLayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaaya

### Code Llama observation

- Model inference lead to OOM with 500 tokens request, when loaded with bfloat16

- Model inference worked with 500 Tokens, with quant_config done with 4-bit.

- In quantisation 1GB of Vram is consumed for inference


- **The model out was gibberish**

- Requested for the code related to dictionary

- Even after reviewing the generation configs, only gibberish was generated